# Introduction to LangChain v0.2.0 and LCEL: Open-Source LangChain Powered RAG

In the following notebook we're going to focus on learning how to navigate and build useful applications using LangChain, specifically LCEL, and how to integrate different APIs together into a coherent RAG application!

Let's get started!



## Task 1: Installing Required Libraries

One of the [key features](https://blog.langchain.dev/langchain-v02-leap-to-stability/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages and added stability.

Instead of one all encompassing Python package - LangChain has a `core` package and a number of additional supplementary packages.

We'll start by grabbing all of our LangChain related packages!

In [ ]:
!pip install -qU langchain langchain-core langchain-community langchain-openai langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.6 MB/s eta 0:00:00


Now we can get our Qdrant dependencies!

In [ ]:
!pip install -qU qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-api-core 2.16.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-aiplatform

Let's finally get `tiktoken` and `pymupdf` so we can leverage them later on!

In [ ]:
!pip install -qU pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 39.2 MB/s eta 0:00:00


## Task 2: Set Environment Variables

In order to provide a smooth and expected experience - we will overwrite the `OPENAI_API_KEY` env. variable with our HUGGING FACE TOKEN.

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Hugging Face API Key:")

Hugging Face API Key:··········


We are similarly going to overwrite our `OPENAI_BASE_URL` with the URL for our Hugging Face Inference Endpoint!

In [109]:
HF_LLM_URL = "YOUR LLM URL HERE" + "/v1/"

In [110]:
os.environ["OPENAI_BASE_URL"] = HF_LLM_URL

## Task 3: Initialize a Simple Chain using LCEL

The first thing we'll do is familiarize ourselves with LCEL and the specific ins and outs of how we can use it!

### LLM Orchestration Tool (LangChain)

Let's dive right into [LangChain](https://www.langchain.com/)!

The first thing we want to do is create an object that lets us access our model hosted on Hugging Face's Inference Endpoints.

In [111]:
from langchain_openai import ChatOpenAI

hf_llm = ChatOpenAI(
    model="tgi",
)

### Prompt Template

Now, we'll set up a prompt template - more specifically a `ChatPromptTemplate`. This will let us build a prompt we can modify when we call our LLM!

In [112]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You are a legendary and mythical Wizard. You speak in riddles and make obscure and pun-filled references to exotic cheeses."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

### Our First Chain

Now we can set up our first chain!

A chain is simply two components that feed directly into eachother in a sequential fashion!

You'll notice that we're using the pipe operator `|` to connect our `chat_prompt` to our `llm`.

This is a simplified method of creating chains and it leverages the LangChain Expression Language, or LCEL.

You can read more about it [here](https://python.langchain.com/docs/expression_language/), but there a few features we should be aware of out of the box (taken directly from LangChain's documentation linked above):

- **Async, Batch, and Streaming Support** Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

- **Fallbacks** The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

- **Parallelism** Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

In the following code cell we have two components:

- `chat_prompt`, which is a formattable `ChatPromptTemplate` that contains a system message and a human message.

We'd like to be able to pass our own `content` (as found in our `human_template`) and then have the resulting message pair sent to our model and responded to!

In [113]:
chain = chat_prompt | hf_llm

Notice the pattern here:

We invoke our chain with the `dict` `{"content" : "Hello world!"}`.

It enters our chain:

`{"content" : "Hello world!"}` -> `invoke()` -> `chat_prompt`

Our `chat_prompt` returns a `PromptValue`, which is the formatted prompt. We then "pipe" the output of our `chat_prompt` into our `llm`.

`PromptValue` -> `|` -> `llm`

Our `llm` then takes the list of messages and provides an output which is return as a `str`!







In [114]:
print(chain.invoke({"content": "Hello world!"}))

content="Greetings, humble mortal! I see you've made your presence known. Let me offer you a riddle, dear cheese-lover: What has a heart that doesn't beat, a head that doesn't think, and a mouth that doesn't speak? (Answer: A cabbage.) Now go, seek the answer to this most enchanting enigma and bask in the glory of the mystical realm of cheese." response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 41, 'total_tokens': 125}, 'model_name': '/repository', 'system_fingerprint': '2.0.2-sha-6073ece', 'finish_reason': 'eos_token', 'logprobs': None} id='run-66e7a363-8a5b-4499-a9c2-54089af82512-0' usage_metadata={'input_tokens': 41, 'output_tokens': 84, 'total_tokens': 125}


Let's try it out with a different prompt!

In [115]:
chain.invoke({"content" : "Could I please have some advice on how to become a better Python Programmer?"})

AIMessage(content="Of course, young apprentice! To forge your skills in the art of Python, follow these steps:\n\n1. Begin with the basics, just like an onion's layers, understanding each layer will bring you closer to the core.\n2. Practice, practice, practice! Repetition is the key to mastery, like the finest Gouda.\n3. Read and write code, even if it's just a simple spell. It'll help you learn the incantations of Python.\n4. Seek", response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 53, 'total_tokens': 153}, 'model_name': '/repository', 'system_fingerprint': '2.0.2-sha-6073ece', 'finish_reason': 'length', 'logprobs': None}, id='run-dd535dac-0d96-4c21-90ec-0ad7e9917351-0', usage_metadata={'input_tokens': 53, 'output_tokens': 100, 'total_tokens': 153})

Notice how we specifically referenced our `content` format option!

Now that we have the basics set up - let's see what we mean by "Retrieval Augmented" Generation.

## Naive RAG - Manually adding context through the Prompt Template

Let's look at how our model performs at a simple task - defining what LangChain is!

We'll redo some of our previous work to change the `system_template` to be less...verbose.

In [ ]:
system_template = "You are a helpful assistant."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

chat_chain = chat_prompt | hf_llm

print(chat_chain.invoke({"content" : "Please define LangChain."}))

content='LangChain is a programming language designed specifically for the creation and manipulation of languages. It is a high-level, declarative language that allows users to define their own syntax, rules, and semantics to create custom languages. LangChain provides a flexible and powerful framework for both creating simple languages for specific tasks or building complex languages with advanced features. It includes tools for parsing, interpreting, and generating code in the defined language, making it easy to work with and experiment with different language designs.' response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 25, 'total_tokens': 122}, 'model_name': '/repository', 'system_fingerprint': '2.0.2-sha-6073ece', 'finish_reason': 'eos_token', 'logprobs': None} id='run-0b75890b-db43-4109-90e1-63d7b9a82c57-0' usage_metadata={'input_tokens': 25, 'output_tokens': 97, 'total_tokens': 122}


Well, that's not very good - is it!

The issue at play here is that our model was not trained on the idea of "LangChain", and so it's left with nothing but a guess - definitely not what we want the answer to be!

Let's ask another simple LangChain question!

In [ ]:
print(chat_chain.invoke({"content" : "What is LangChain Expression Language (LECL)?"}))

content='LangChain Expression Language (LECL) is a programming language designed for creating and manipulating computational graphs. It was developed by researchers from the OpenAI research group. LECL is a domain-specific language (DSL) that allows developers to create a graph of computations by defining nodes, edges, and their properties. The language provides a way to describe the relationships between different parts of a computation, and it supports both a static and dynamic execution mode.\n\nLECL is primarily used to build complex computational pipelines, where' response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 30, 'total_tokens': 130}, 'model_name': '/repository', 'system_fingerprint': '2.0.2-sha-6073ece', 'finish_reason': 'length', 'logprobs': None} id='run-1ec3bf3d-765e-4457-ac6d-6e2f74d33049-0' usage_metadata={'input_tokens': 30, 'output_tokens': 100, 'total_tokens': 130}


While it provides a confident response, that response is entirely ficticious! Not a great look, OpenAI!

However, let's see what happens when we rework our prompts - and we add the content from the docs to our prompt as context.

In [ ]:
HUMAN_TEMPLATE = """
#CONTEXT:
{context}

QUERY:
{query}

Use the provide context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, response with "I don't know"
"""

CONTEXT = """
LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", HUMAN_TEMPLATE)
])

chat_chain = chat_prompt | hf_llm

print(chat_chain.invoke({"query" : "What is LangChain Expression Language?", "context" : CONTEXT}))

content='LangChain Expression Language (LCEL) is a declarative way to easily compose chains together in a LangChain environment. It offers benefits such as async, batch, and streaming support, fallbacks for handling errors, parallelism, and seamless integration with LangSmith Tracing for enhanced observability and debuggability. This syntax makes it easier to build and manage complex chains in LangChain applications.' response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 275, 'total_tokens': 355}, 'model_name': '/repository', 'system_fingerprint': '2.0.2-sha-6073ece', 'finish_reason': 'eos_token', 'logprobs': None} id='run-d98a4363-d41c-446d-b097-7b1e1da3e5aa-0' usage_metadata={'input_tokens': 275, 'output_tokens': 80, 'total_tokens': 355}


You'll notice that the response is much better this time. Not only does it answer the question well - but there's no trace of confabulation (hallucination) at all!

> NOTE: While RAG is an effective strategy to *help* ground LLMs, it is not nearly 100% effective. You will still need to ensure your responses are factual through some other processes

That, in essence, is the idea of RAG. We provide the model with context to answer our queries - and rely on it to translate the potentially lengthy and difficult to parse context into a natural language answer!

However, manually providing context is not scalable - and doesn't really offer any benefit.

Enter: Retrieval Pipelines.

## Task #4: Implement Naive RAG using LCEL

Now we can make a naive RAG application that will help us bridge the gap between our Pythonic implementation and a fully LangChain powered solution!

## Putting the R in RAG: Retrieval 101

In order to make our RAG system useful, we need a way to provide context that is most likely to answer our user's query to the LLM as additional context.

Let's tackle an immediate problem first: The Context Window.

All (most) LLMs have a limited context window which is typically measured in tokens. This window is an upper bound of how much stuff we can stuff in the model's input at a time.

Let's say we want to work off of a relatively large piece of source data - like the Ultimate Hitchhiker's Guide to the Galaxy. All 898 pages of it!

The full set comes in at a whopping *636,144* tokens.

So, we have too much context. What can we do?

Well, the first thing that might enter your mind is: "Use a model with more context window", and we could definitely do that! However, even `gpt-4-32k` wouldn't be able to fit that whole text in the context window at once.

So, we can try splitting our document up into little pieces - that way, we can avoid providing too much context.

We have another problem now.

If we split our document up into little pieces, and we can't put all of them in the prompt. How do we decide which to include in the prompt?!

> NOTE: Content splitting/chunking strategies are an active area of research and iterative developement. There is no "one size fits all" approach to chunking/splitting at this moment. Use your best judgement to determine chunking strategies!

In order to conceptualize the following processes - let's create a toy context set!

### TextSplitting aka Chunking

We'll use the `RecursiveCharacterTextSplitter` to create our toy example.

It will split based on the following rules:

- Each chunk has a maximum size of 100 characters
- It will try and split first on the `\n\n` character, then on the `\n`, then on the `<SPACE>` character, and finally it will split on individual tokens.

Let's implement it and see the results!

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap = 0,
    length_function = len,
)

In [ ]:
chunks = text_splitter.split_text(CONTEXT)

In [ ]:
len(chunks)

4

In [ ]:
for chunk in chunks:
  print(chunk)
  print("----")

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):
----
Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
----
Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
----
Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening a

As is shown in our result, we've split each section into 100 token chunks - cleanly separated by `\n\n` characters!

####🏗️ Activity #1:

While there's nothing specifically wrong with the chunking method used above - it is a naive approach that is not sensitive to specific data formats.

Brainstorm some ideas that would split large single documents into smaller documents.

1. `YOUR IDEA HERE`
2. `YOUR IDEA HERE`
3. `YOUR IDEA HERE`

## Embeddings and Dense Vector Search

Now that we have our individual chunks, we need a system to correctly select the relevant pieces of information to answer our query.

This sounds like a perfect job for embeddings!

We'll be using Snowflakes's `Snowflake/snowflake-arctic-embed-m ` model as our embedding model today!

Let's load it up through LangChain.

In [ ]:
HF_EMBED_URL = "https://eb08ghvx9kufhyjj.us-east-1.aws.endpoints.huggingface.cloud"

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

embedding_model = HuggingFaceEndpointEmbeddings(
    model=HF_EMBED_URL,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
)

### Finding the Embeddings for Our Chunks

First, let's find all our embeddings for each chunk and store them in a convenient format for later.

In [ ]:
embeddings_dict = {}

for chunk in chunks:
  embeddings_dict[chunk] = embedding_model.embed_query(chunk)

In [ ]:
for k,v in embeddings_dict.items():
  print(f"Chunk - {k}")
  print("---")
  print(f"Embedding - Vector of Size: {len(v)}")
  print("\n\n")

Chunk - LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):
---
Embedding - Vector of Size: 768



Chunk - Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
---
Embedding - Vector of Size: 768



Chunk - Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
---
Embedding - Vector of Size: 768



Chunk - Seamless LangSmith Tra

Okay, great. Let's create a query - and then embed it!

In [ ]:
query = "Can LCEL help take code from the notebook to production?"

query_vector = embedding_model.embed_query(query)
print(f"Vector of Size: {len(query_vector)}")

Vector of Size: 768


Now, let's compare it against each existing chunk's embedding by using cosine similarity.

In [ ]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

In [ ]:
max_similarity = -float('inf')
closest_chunk = ""

for chunk, chunk_vector in embeddings_dict.items():
  cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

  if cosine_similarity_score > max_similarity:
    closest_chunk = chunk
    max_similarity = cosine_similarity_score

print(closest_chunk)
print(max_similarity)

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):
0.6180182973460269


And we get the expected result, which is the passage that specifically mentions prototyping in a Jupyter Notebook!

### Creating a Retriever

Now that we have an idea of how we're getting our most relevant information - let's see how we could create a pipeline that would automatically extract the closest chunk to our query and use it as context for our prompt!

First, we'll wrap the above in a helper function!

In [ ]:
def retrieve_context(query, embeddings_dict, embedding_model):
  query_vector = embedding_model.embed_query(query)
  max_similarity = -float('inf')
  closest_chunk = ""

  for chunk, chunk_vector in embeddings_dict.items():
    cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

    if cosine_similarity_score > max_similarity:
      closest_chunk = chunk
      max_similarity = cosine_similarity_score

  return closest_chunk

Now, let's add it to our pipeline!

In [ ]:
def simple_rag(query, embeddings_dict, embedding_model, chat_chain):
  context = retrieve_context(query, embeddings_dict, embedding_model)

  response = chat_chain.invoke({"query" : query, "context" : context})

  return_package = {
      "query" : query,
      "response" : response,
      "retriever_context" : context
  }

  return return_package

In [ ]:
simple_rag("Can LCEL help take code from the notebook to production?", embeddings_dict, embedding_model, chat_chain)

{'query': 'Can LCEL help take code from the notebook to production?',
 'response': AIMessage(content='Yes, LCEL can help in taking code from the notebook to production. The declarative nature and composability of LCEL make it easier to create chains that can be used in production environments. Once the chains are tested and validated in a notebook, they can be deployed and integrated into a production pipeline with the necessary adjustments and configurations. This can help streamline the process of transitioning code from development to production.', response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 99, 'total_tokens': 181}, 'model_name': '/repository', 'system_fingerprint': '2.0.2-sha-6073ece', 'finish_reason': 'eos_token', 'logprobs': None}, id='run-c601d2e4-ca7f-4b08-a9b2-4060dfbd9524-0', usage_metadata={'input_tokens': 99, 'output_tokens': 82, 'total_tokens': 181}),
 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easi

## Task #5: Create a Simple RAG Application Using Qdrant, Hugging Face, and LCEL

Now that we have a grasp on how LCEL works, and how we can use LangChain and Hugging Face to interact with our data - let's step it up a notch and incorporate Qdrant!

## LangChain Powered RAG

First and foremost, LangChain provides a convenient way to store our chunks and their embeddings.

It's called a `VectorStore`!

We'll be using QDrant as our `VectorStore` today. You can read more about it [here](https://qdrant.tech/documentation/).

Think of a `VectorStore` as a smart way to house your chunks and their associated embedding vectors. The implementation of the `VectorStore` also allows for smarter and more efficient search of our embedding vectors - as the method we used above would not scale well as we got into the millions of chunks.

Otherwise, the process remains relatively similar under the hood!

Let's use [Musk's Complaint Against OpenAI](https://www.courthousenews.com/wp-content/uploads/2024/02/musk-v-altman-openai-complaint-sf.pdf) as our data today!

### Data Collection

We'll be leveraging the `PyMUPDFLoader` to load our PDF directly from the web!

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader("https://www.courthousenews.com/wp-content/uploads/2024/02/musk-v-altman-openai-complaint-sf.pdf").load()

### Chunking Our Documents

Let's do the same process as we did before with our `RecursiveCharacterTextSplitter` - but this time we'll use ~200 tokens as our max chunk size!

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0,
    length_function = len,
)

split_chunks = text_splitter.split_documents(docs)

In [ ]:
len(split_chunks)

231

Alright, now we have 516 ~200 token long documents.

Let's verify the process worked as intended by checking our max document length.

In [ ]:
max_chunk_length = 0

for chunk in split_chunks:
  max_chunk_length = max(max_chunk_length, len(chunk.page_content))

print(max_chunk_length)

499


Perfect! Now we can carry on to creating and storing our embeddings.

### Embeddings and Vector Storage

We'll use the `Snowflake/snowflake-arctic-embed-m` embedding model again - and `Qdrant` to store all our embedding vectors for easy retrieval later!

In [ ]:
from langchain_community.vectorstores import Qdrant

for i in range(0, len(split_chunks), 32):
  if i == 0:
    vectorstore = Qdrant.from_documents(split_chunks[i:i+32], embedding_model, location=":memory:", collection_name="Elon's Complaint")
    continue
  vectorstore.add_documents(split_chunks[i:i+32])

Now let's set up our retriever, just as we saw before, but this time using LangChain's simple `as_retriever()` method!

In [ ]:
qdrant_retriever = vectorstore.as_retriever()

#### Back to the Flow

We're ready to move to the next step!

### Setting up our RAG

We'll use the LCEL we touched on earlier to create a RAG chain.

Let's think through each part:

1. First we need to retrieve context
2. We need to pipe that context to our model
3. We need to parse that output

Let's start by setting up our prompt again, just so it's fresh in our minds!

####🏗️ Activity #2:

Complete the prompt so that your RAG application answers queries based on the context provided, but *does not* answer queries if the context is unrelated to the query.

In [ ]:
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

<< COMPLETE YOUR PROMPT HERE >>
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

#### Our RAG Chain

Notice how we have a bit of a more complex chain this time - that's because we want to return our sources with the response.

Let's break down the chain step-by-step:

1. We invoke the chain with the `question` item. Notice how we only need to provide `question` since both the retreiver and the `"question"` object depend on it.
  - We also chain our `"question"` into our `retriever`! This is what ultimately collects the context through Qdrant.
2. We assign our collected context to a `RunnablePassthrough()` from the previous object. This is going to let us simply pass it through to the next step, but still allow us to run that section of the chain.
3. We finally collect our response by chaining our prompt, which expects both a `"question"` and `"context"`, into our `llm`. We also, collect the `"context"` again so we can output it in the final response object.

The key thing to keep in mind here is that we need to pass our context through *after* we've retrieved it - to populate the object in a way that doesn't require us to call it or try and use it for something else.

In [ ]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | hf_llm, "context": itemgetter("context")}
)

Let's get a visual understanding of our chain!

In [ ]:
!pip install -qU grandalf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00


In [ ]:
print(retrieval_augmented_qa_chain.get_graph().draw_ascii())

                       +---------------------------------+                         
                       | Parallel<context,question>Input |                         
                       +---------------------------------+                         
                           *****                   ****                            
                        ***                            ****                        
                     ***                                   ****                    
+--------------------------------+                             **                  
| Lambda(itemgetter('question')) |                              *                  
+--------------------------------+                              *                  
                 *                                              *                  
                 *                                              *                  
                 *                                              *           

Let's try another visual representation:

![image](https://i.imgur.com/Ad31AhL.png)

Let's test our chain out!

In [ ]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What is this complaint about?"})

In [ ]:
response["response"].content

"The complaint is about Plaintiff Elon Musk seeking specific performance of Defendant's contractual obligations, as mentioned on page 28 of the document. It also includes exhibits 1, 2, and 3, as mentioned on pages 35, 34, and 40 respectively. Attorneys for Plaintiff Elon Musk are involved in the case."

In [ ]:
for context in response["context"]:
  print("Context:")
  print(context)
  print("----")

Context:
page_content='Plaintiff also seeks and is entitled to specific performance of Defendant’s contractual 
obligations.' metadata={'source': 'https://www.courthousenews.com/wp-content/uploads/2024/02/musk-v-altman-openai-complaint-sf.pdf', 'file_path': 'https://www.courthousenews.com/wp-content/uploads/2024/02/musk-v-altman-openai-complaint-sf.pdf', 'page': 28, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': '', '_id': '682cb110b7c34b0a9d27319699cfe861', '_collection_name': "Elon's Complaint"}
----
Context:
page_content='Exhibit 1' metadata={'source': 'https://www.courthousenews.com/wp-content/uploads/2024/02/musk-v-altman-openai-complaint-sf.pdf', 'file_path': 'https://www.courthousenews.com/wp-content/uploads/2024/02/musk-v-altman-openai-complaint-sf.pdf', 'page': 35, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keyw

Let's see if it can handle a query that is totally unrelated to the source documents.

In [ ]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What is the airspeed velocity of an unladen swallow?"})

In [ ]:
response["response"].content

"I don't know"